### Univariate MLP Models
1. Data Preparation
2. MLP Model

In [1]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [2]:
# split a univariate sequence into samples
def split_sequence(sequence, n_steps):
    X, y = list(), list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the sequence
        if end_ix > len(sequence)-1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

In [3]:
# define input sequence
raw_seq = [10, 20, 30, 40, 50, 60, 70, 80, 90]
# choose a number of time step
n_steps = 3
X, y = split_sequence(raw_seq, n_steps)
print("Actual Input")
print(raw_seq)
print("Reshaped Input")
print(X)
print(X.shape)
print("Desried Output")
print(y)
print(y.shape)

Actual Input
[10, 20, 30, 40, 50, 60, 70, 80, 90]
Reshaped Input
[[10 20 30]
 [20 30 40]
 [30 40 50]
 [40 50 60]
 [50 60 70]
 [60 70 80]]
(6, 3)
Desried Output
[40 50 60 70 80 90]
(6,)


In [4]:
# define model
model = Sequential()
model.add(Dense(units=100, activation='relu', input_dim=n_steps))
model.add(Dense(units=1))
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(x=X, y=y, epochs=2000, verbose=False)

In [5]:
# demonstrate prediction
x_input = np.array([70, 80, 90])
x_input = x_input.reshape(1, n_steps)
print(x_input)
print(x_input.shape)
yhat = model.predict(x_input, verbose=False)
print(yhat)
print(yhat.shape)

[[70 80 90]]
(1, 3)
[[100.87717]]
(1, 1)


### Multivariate multistep MLP Models

In [6]:
# split a multivariate sequence into samples
def split_sequences(sequences, n_steps_in, n_steps_out):
    X, y = list(), list()
    for i in range(len(sequences)):
        # find the end of this pattern
        end_ix = i + n_steps_in
        out_end_ix = end_ix + n_steps_out-1
        # check if we are beyond the dataset
        if out_end_ix > len(sequences):
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1:out_end_ix, -1]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)
# define input sequence
in_seq1 = np.array([10, 20, 30, 40, 50, 60, 70, 80, 90])
in_seq2 = np.array([15, 25, 35, 45, 55, 65, 75, 85, 95])
out_seq = np.array([in_seq1[i]+in_seq2[i] for i in range(len(in_seq1))])
# convert to [rows, columns] structure 
in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))
out_seq = out_seq.reshape((len(out_seq), 1)) 
# horizontally stack columns
dataset = np.hstack((in_seq1, in_seq2, out_seq))
print(dataset)
print(dataset.shape)

[[ 10  15  25]
 [ 20  25  45]
 [ 30  35  65]
 [ 40  45  85]
 [ 50  55 105]
 [ 60  65 125]
 [ 70  75 145]
 [ 80  85 165]
 [ 90  95 185]]
(9, 3)


In [7]:
# choose a number of time steps
n_steps_in, n_steps_out = 3, 2
# convert into input/output
X, y = split_sequences(dataset, n_steps_in, n_steps_out)
# the dataset knows the number of features, e.g. 2
n_features = X.shape[2]
print(X.shape, y.shape)
# summarize the data
for i in range(len(X)):
    print(X[i], y[i])

(6, 3, 2) (6, 2)
[[10 15]
 [20 25]
 [30 35]] [65 85]
[[20 25]
 [30 35]
 [40 45]] [ 85 105]
[[30 35]
 [40 45]
 [50 55]] [105 125]
[[40 45]
 [50 55]
 [60 65]] [125 145]
[[50 55]
 [60 65]
 [70 75]] [145 165]
[[60 65]
 [70 75]
 [80 85]] [165 185]


In [8]:
# flatten input
n_input = X.shape[1] * X.shape[2]
X = X.reshape((X.shape[0], n_input))
print(X)
print(X.shape)

[[10 15 20 25 30 35]
 [20 25 30 35 40 45]
 [30 35 40 45 50 55]
 [40 45 50 55 60 65]
 [50 55 60 65 70 75]
 [60 65 70 75 80 85]]
(6, 6)


In [9]:
# define model
model = Sequential()
model.add(Dense(units=100, activation='relu', input_dim=n_input))
model.add(Dense(units=n_steps_out))
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(x=X, y=y, epochs=2000, verbose=False)

In [10]:
# demonstrate prediction
x_input = np.array([[70, 75], [80, 85], [90, 95]])
x_input = x_input.reshape((1, n_input))
print(x_input)
print(x_input.shape)
yhat = model.predict(x_input, verbose=False)
print(yhat)
print(yhat.shape)

[[70 75 80 85 90 95]]
(1, 6)
[[186.90887 208.57956]]
(1, 2)
